In [ ]:
import pickle
import gzip
import numpy as np
import random
import pickle 
import matplotlib.pyplot as plt
import math

from losses import losses

def load_data():
    with gzip.open("../data/mnist.pkl.gz", "rb") as file:
        train, val, test = pickle.load(file, encoding="latin1")
    return train, val, test

def load_data_wrapper():
    train, val, test = load_data()
    
    train_inputs = [np.reshape(x, (784, 1)) for x in train[0]]
    train_outputs = [one_hot_encode(y, 10) for y in train[1]]
    train_data = list(zip(train_inputs, train_outputs))
    
    val_inputs = [np.reshape(x, (784, 1)) for x in val[0]]
    val_data = list(zip(val_inputs, val[1]))
    
    test_inputs = [np.reshape(x, (784, 1)) for x in test[0]]    
    test_data = list(zip(test_inputs, test[1]))
    
    return train_data, val_data, test_data

def one_hot_encode(y, num_categories):
    one_hot = np.zeros((num_categories, 1))
    one_hot[y] = 1
    return one_hot

In [ ]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
train, val, test = load_data_wrapper()

In [ ]:
class NetworkVectorized(object):
    def __init__(self, sizes, loss=losses.MSELoss()):
        """sizes: list of ints signifying size of each layer, including the input and output layers"""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(next_h, prev_h) 
                        for next_h, prev_h in zip(sizes[1:], sizes[:-1])]
        self.loss = loss
    
    def feedforward(self, a, return_activations=False):
        """roughly 3 times faster than for-loop version"""
        activations = [a]
        zs = [] 
        
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, a)+b
            a = sigmoid(z)
            if return_activations:
                activations.append(a)
                zs.append(z)
        if return_activations:
            return a, (activations, zs)
        return a
    
    
    def SGD(self, train_data, epochs, batch_size, learning_rate, test_data=None):
        if test_data:
            n_test = len(test_data)
        n_train = len(train_data)
        
        for i in range(epochs):
            random.shuffle(train_data)
            mini_batches = [train_data[j:j+batch_size] for j in range(0, n_train, batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, learning_rate)
            
            if test_data:
                correct_preds = self.evaluate(test_data)
                print(f"Epoch {i}: accuracy: {correct_preds} / {n_test}")
                
            else:
                print(f"Epoch {i}: complete")
                
                
    def update_mini_batch(self, mini_batch, learning_rate):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        x = np.asarray([_x.ravel() for _x, _y in mini_batch]).transpose()
        y = np.asarray([_y.ravel() for _x, _y in mini_batch]).transpose()
        
        delta_nabla_b, delta_nabla_w = self.backprop(x, y)
        
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b,delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w,delta_nabla_w)]
        
        self.weights = [w -(learning_rate/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]    
        self.biases = [b -(learning_rate/len(mini_batch))*nb 
                        for b, nb in zip(self.biases, nabla_b)]
        
        
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Forward pass 
        activations, zs = self.feedforward(x, return_activations=True)[1]

            
        # Backward pass 
        dloss = self.loss.backward(activations[-1], y)
        dz = dloss * sigmoid_derivative(zs[-1])
        nabla_b[-1] = np.sum(dz, axis=1, keepdims=True)
        nabla_w[-1] = dz.dot(activations[-2].T)
            
        for l in range(2, self.num_layers):
            z = zs[-l]
            d_sigmoid = sigmoid_derivative(z)
            dz = self.weights[-l+1].T.dot(dz) * d_sigmoid
            nabla_b[-l] = np.sum(dz, axis=1, keepdims=True)
            nabla_w[-l] = dz.dot(activations[-l-1].T)
        
        return nabla_b, nabla_w
    
    
    def evaluate(self, test_data):
        x = np.asarray([x.ravel() for x, _ in test_data]).T
        y = np.asarray([_y.ravel() for _x, _y in test_data])
        
        y_hat = self.feedforward(x)
        preds = np.expand_dims(np.argmax(y_hat, axis=0), axis=1)
        
        correct_preds = sum(y == preds)[0]
        
        return correct_preds
    
    
    def get_activations_for_example(self, x):
        activations = [x]
        a = x
        for w, b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w, a)+b)
            activations.append(a)
        return activations
    
    def plot_first_layer_weights(self, name=None):
        fig=plt.figure(figsize=(8,8))
        w_h, w_w = net.weights[0].shape
        fig_cols = math.floor(math.sqrt(w_h))
        fig_rows = math.ceil(w_h/fig_cols)
                
        for i in range(w_h):
            img = np.reshape(self.weights[0][i], (28, 28))    
            fig.add_subplot(fig_rows, fig_cols, i+1)
            plt.imshow(img, interpolation='nearest')
        fig.suptitle(f'First Layer Weights', fontsize=20)
        
        if name:
            fig.savefig(f"{name}.png")

## Training

In [ ]:
net = NetworkVectorized([784, 30, 10], loss=losses.CrossEntropyLoss())

In [ ]:
net.SGD(train_data=train, epochs=30, batch_size=20, learning_rate=3.0, test_data=test)

In [ ]:
net.plot_first_layer_weights(name="CrossEntropyLoss")

In [ ]:
net2 = NetworkVectorized([784, 30, 10], loss=losses.MSELoss())
net2.SGD(train_data=train, epochs=30, batch_size=20, learning_rate=3.0, test_data=test)

In [ ]:
net2.plot_first_layer_weights(name="MSEloss")